# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.policy import MedianStoppingPolicy
from azureml.train.hyperdrive import uniform, choice
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Getting available workspace
ws = Workspace.from_config()

experiment_name = 'my-hyperdrive-project'

exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

SDK version: 1.20.0
Workspace name: quick-starts-ws-136595
Azure region: southcentralus
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-136595


In [3]:
## Creating compute cluster
cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [34]:
# Logistic Regression needs 2 parameters - 
# C = Regularization Strength (float)
# max_iter = Max iterations (int)
# We use Random Parameter Sampling initially as it supports discrete as well as continuous parameters
# We will try Bayesian Sampling later
# Check initial values for parameters. Current values are used for trial

## Create different params that will be used during training
param_sampling = RandomParameterSampling({
    "C": uniform(1,5),
    "max_iter": choice(range(1, 100))
})

primary_metric_name = "Accuracy"
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE

## Specify early termination policy
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

## Create a SKLearn estimator for use with train.py
estimator = SKLearn(source_directory = '.',
              entry_script = 'train.py',
              compute_target = compute_target)

## Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                     primary_metric_name=primary_metric_name,
                                     primary_metric_goal=primary_metric_goal,
                                     estimator=estimator,
                                     policy=early_termination_policy,
                                     max_total_runs=2)


In [27]:
import pandas as pd

print("Reading data from CSV file")
dataframe = pd.read_csv('./breast_cancer_dataset.csv')
dataframe.head()


Reading data from CSV file


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [32]:
## Submit the hyperdrive run to the experiment and show run details with the widget
ws.write_config(path='.azureml/')

hyperdrive_run = exp.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [33]:
## Use RunDetails widget to show details of the experiment in progress
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == 'Completed')

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_3d805478-45cc-4814-9925-fa3e786ca635
Web View: https://ml.azure.com/experiments/my-hyperdrive-project/runs/HD_3d805478-45cc-4814-9925-fa3e786ca635?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-136595/workspaces/quick-starts-ws-136595

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-30T16:37:04.268171][API][INFO]Experiment created<END>\n"<START>[2021-01-30T16:37:05.8464459Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-30T16:37:06.997225][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space<END>\n""<START>[2021-01-30T16:37:07.137670][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_3d805478-45cc-4814-9925-fa3e786ca635
Web View: https://ml.azure.com/experiments/my-hyperdrive-project/runs/HD_3d805478-45cc-4814-9925-fa3e786ca635?wsid=/subscri

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User errors were found in at least one of the child runs.",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"User errors were found in at least one of the child runs.\",\n        \"messageParameters\": {},\n        \"details\": []\n    },\n    \"time\": \"0001-01-01T00:00:00.000Z\"\n}"
    }
}

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [18]:
## Get the best run and save the model from that run
hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()
hyperdrive_best_run_mertics = hyperdrive_best_run.get_metrics()
parameter_values = hyperdrive_best_run.get_details()['runDefinition']['arguments']
print('best run id : ', hyperdrive_best_run.id)
print('best run accuracy :', hyperdrive_best_run_mertics['Accuracy'])
print('C : ', parameter_values[1])
print('max_iter : ', parameter_values[3])

## Save the best model
best_run_model_name = 'Udacity-hyperdrive-model'
best_run_model_path = './outputs/model.joblib'
hyperdrive_best_run.download_file('outputs/model.joblib',best_run_model_path)
model = hyperdrive_best_run.register_model(model_name=best_run_model_name,model_path=best_run_model_path)


best run id :  HD_4c325573-7b84-4496-85be-3ab6ecb8c762_1
best run accuracy : 0.9151186315983079
C :  4.90624680121264
max_iter :  82


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service